# PyTerrier Notebook for Full-Rank Submissions

This notebook serves as a baseline full-rank submission for [TIRA](https://tira.io)/[TIREx](https://tira.io/tirex) that builds a PyTerrier index and subsequently creates a run with BM25.

### Step 1: Ensure Libraries are Imported

In [1]:
import os

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !pip3 install  python-terrier tira==0.0.88 ir_datasets
    !pip3 install -q python-terrier
    !pip3 install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
else:
    print('We are in the TIRA sandbox.')

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

#import matplotlib.pyplot as plt

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ensure Pyterrier integration is loaded

In [2]:
ensure_pyterrier_is_loaded()

Due to execution in TIRA, I have patched ir_datasets to always return the single input dataset mounted to the sandbox.
Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Step 2: Load data, create index

In [3]:
dataset = pt.get_dataset('irds:ir-lab-jena-leipzig-wise-2023/validation-20231104-training')
qrels = dataset.get_qrels()
topics = dataset.get_topics(variant="title")
topics = topics.head(5)

index_loc = "./index1"
indexer = pt.IterDictIndexer(index_loc)
indexref = indexer.index(dataset.get_corpus_iter())

Load ir_dataset "ir-lab-jena-leipzig-wise-2023/validation-20231104-training" from tira.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


ir-lab-jena-leipzig-wise-2023/validation-20231104-training documents:   0%|          | 0/61307 [00:00<?, ?it/s]

ValueError: Index already exists at ./index1/data.properties

#### For retrieval with PL2 c=1.2 seems to work best

In [ ]:
pl2 = pt.BatchRetrieve(indexer, wmodel="PL2", verbose=True, controls={"b" : 1.2})

#### Next, we want to experiment with query expansion for retrieval with BM25.

In [ ]:
bm25 = pt.BatchRetrieve(indexer, wmodel="BM25", verbose=True, controls={"b" : 0.8})
#bm25= ~bm25

bo1_expansion = ~bm25 >> pt.rewrite.Bo1QueryExpansion(indexer)
bm25_bo1 = bo1_expansion >> bm25

kl = ~bm25 >> pt.rewrite.KLQueryExpansion(indexer)
bm25_kl = kl >> bm25

In [ ]:
run = bm25_bo1(topics)

BR(BM25): 100%|██████████| 5/5 [00:00<00:00,  9.22q/s]


In [ ]:
run_bm25  = bm25(topics)

BR(BM25): 100%|██████████| 5/5 [00:00<00:00, 15.16q/s]


In [ ]:
run_bm25.head()

,qid,docid,docno,rank,score,query
0,q072224,17325,doc072205301584,0,11.046623,purchase money
1,q072224,59628,doc072202002905,1,11.038193,purchase money
2,q072224,27801,doc072211705166,2,10.783108,purchase money
3,q072224,58906,doc072214404676,3,10.725315,purchase money
4,q072224,37201,doc072201705080,4,10.705278,purchase money


In [ ]:
dataset.get_topics()

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,qid,text,title,query,description,narrative
0,q072224,purchase money,purchase money,purchase money,None,None
1,q072226,purchase used car,purchase used car,purchase used car,None,None
2,q072232,buy gold silver,buy gold silver,buy gold silver,None,None
3,q072240,adenovirus,adenovirus,adenovirus,None,None
4,q072242,Water softener,Water softener,water softener,None,None
...,...,...,...,...,...,...
877,q072229958,video twitter,video twitter,video twitter,None,None
878,q072230046,used car,used car,used car,None,None
879,q072230066,used electric car,used electric car,used electric car,None,None
880,q072230072,collector cars,collector cars,collector cars,None,None


In [ ]:
run.head().rename(columns= {"query": "query_0", "query_0": "query"})

,qid,docid,docno,rank,score,query,query_0
0,q072224,59628,doc072202002905,0,14.392958,purchase money,applypipeline:off purchas^1.124916191 monei^1....
1,q072224,27801,doc072211705166,1,12.665061,purchase money,applypipeline:off purchas^1.124916191 monei^1....
2,q072224,16790,doc072215502070,2,12.585387,purchase money,applypipeline:off purchas^1.124916191 monei^1....
3,q072224,19718,doc072211305429,3,12.585387,purchase money,applypipeline:off purchas^1.124916191 monei^1....
4,q072224,40315,doc072203203007,4,12.585387,purchase money,applypipeline:off purchas^1.124916191 monei^1....


#### Additionally, we linearly combine retrieval with BM25 and PL2.

In [ ]:
bm25_bo1_pl2 = (2* bm25_bo1 + pl2)
bm25_kl_pl2 = (2* bm25_kl + pl2).transform(topics)

BR(PL2): 100%|██████████| 5/5 [00:00<00:00, 24.77q/s]


#### Let's run an experiment, to see which retrieval model works best.

#### The combination of BM25 with Bo1 query expansion and PL2 performs slightly better than the others. 

### Next, we want to rerank the output with a transformer.

In [ ]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker()

/home/codespace/.python/current/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. I

In [ ]:
import pandas as pd

corpus = pd.DataFrame(dataset.get_corpus_iter())


class GetText(pt.Transformer):
    def transform(self, topics_or_res: pd.DataFrame) -> pd.DataFrame:
        return pd.merge(topics_or_res, corpus, on="docno")

class ResetQueryColumn(pt.Transformer):
    def transform(self, topics_or_res: pd.DataFrame) -> pd.DataFrame:
        return topics_or_res.rename(columns= {"query": "query_0", "query_0": "query"})

No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


ir-lab-jena-leipzig-wise-2023/validation-20231104-training documents:   5%|▌         | 3335/61307 [00:00<00:03, 17155.32it/s]

No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


ir-lab-jena-leipzig-wise-2023/validation-20231104-training documents: 100%|██████████| 61307/61307 [00:03<00:00, 17143.70it/s]


In [ ]:
from pyterrier_t5 import T5Tokenizer

Rerank with transformer

In [ ]:

bm25_bo1_pl2_monot5 = (bm25_bo1 % 10 >> GetText() >> ResetQueryColumn()
        >> pt.text.sliding(length=400, stride=64, prepend_attr=None, text_attr="text")
        >> monoT5
        >> pt.text.mean_passage())

In [ ]:
print('Create run')
run = bm25_bo1_pl2_monot5(topics)
print('Done, run was created')

Create run


BR(BM25): 100%|██████████| 5/5 [00:00<00:00, 16.27q/s]


calling sliding on df of 50 rows


monoT5:   0%|          | 0/107 [00:00<?, ?batches/s]Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
pt.Experiment(
    [bm25_bo1_pl2, bm25_bo1_pl2_monot5], 
    topics, dataset.get_qrels(), 
    eval_metrics=["ndcg", "map", "recip_rank", "P_10", "recall_10"],
    names=["BM25 >> Bo1 >> BM25", "BM25 >> Bo1 >> BM25 >> MonoT5"],
    baseline = 0
)

BR(BM25): 100%|██████████| 5/5 [00:00<00:00, 55.72q/s]


calling sliding on df of 150 rows


monoT5: 100%|██████████| 251/251 [25:18<00:00,  6.05s/batches]


,name,map,recip_rank,P_10,recall_10,ndcg,map +,map -,map p-value,recip_rank +,...,recip_rank p-value,P_10 +,P_10 -,P_10 p-value,recall_10 +,recall_10 -,recall_10 p-value,ndcg +,ndcg -,ndcg p-value
0,BM25 >> Bo1 >> BM25,0.073471,0.067629,0.08,0.220000,0.226653,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25 >> Bo1 >> BM25 >> MonoT5,0.148178,0.322222,0.10,0.286667,0.227390,2.0,3.0,0.241897,3.0,...,0.235471,1.0,0.0,0.373901,1.0,0.0,0.373901,2.0,3.0,0.99192


### Step 4: Persist run.

In [ ]:
persist_and_normalize_run(run, 't5-reranker')

I use the environment variable "TIRA_OUTPUT_DIR" to determine where I should store the run file using "." as default.
Done. run file is stored under "./run.txt".
